<a href="https://colab.research.google.com/github/Himanshukhanduri20/PySpark/blob/main/new_last_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
!pip install -q findspark
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-06-25 10:10:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  58.7MB/s    in 0.2s    

2022-06-25 10:10:51 (58.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [34]:
df = spark.read.format("text").load("/content/sample_data/example_log.log")

In [35]:
df.show()

+--------------------+
|               value|
+--------------------+
|<166>Jun  1 07:33...|
|                    |
|<166>Jun  2 07:39...|
|                    |
|<166>Jun  3 07:44...|
|                    |
|<166>Jun  4 07:50...|
|                    |
|<166>Jun  5 07:56...|
|                    |
|<166>Jun  6 08:01...|
|                    |
|<166>Jun  7 08:07...|
|                    |
|<166>Jun  8 08:13...|
|                    |
|<166>Jun  9 08:18...|
|                    |
|<166>Jun  10 08:2...|
+--------------------+



In [37]:
df=df.filter(df.value.contains("Trend Micro"))
df.show()

+--------------------+
|               value|
+--------------------+
|<166>Jun  1 07:33...|
|<166>Jun  2 07:39...|
|<166>Jun  3 07:44...|
|<166>Jun  4 07:50...|
|<166>Jun  5 07:56...|
|<166>Jun  6 08:01...|
|<166>Jun  8 08:13...|
|<166>Jun  9 08:18...|
|<166>Jun  10 08:2...|
+--------------------+



In [38]:
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.sql.functions import col

In [41]:

df1=df.withColumn("current_date",F.current_date())
df1=df1.withColumn("year",F.regexp_extract(df1.current_date,"([\\d]+)",1)).drop(df1.current_date)
df1=df1.withColumn("_c6",F.regexp_extract(df.value,'(>[a-zA-Z]+\s*\d+\s*[\d+:]+)',1))
df1=df1.withColumn("Time",F.regexp_extract(df1._c6,'(\d+:\d+:\d+)',1))
df1=df1.withColumn('_c6', F.regexp_replace('_c6', '>', '')).withColumn('Date', F.regexp_replace('_c6', '(\d+:\d+:\d+)', '')).withColumn('Data_source_host_name', F.regexp_extract(df1.value, '([a-zA-Z]+[-][a-zA-Z]+[-][a-zA-Z]+[-]\d+)',1))
df1=df1.withColumn("Day",F.regexp_extract(df1.Date,'(\d+)',1))
df1=df1.withColumn("Month",F.regexp_extract(df1.Date,'([a-zA-Z]+)',1))
df1=df1.withColumn("Month", when(col("month") == "Jan", "01").when(col("month") == "Feb", "02").when(col("month") == "Mar", "03").when(col("month") == "Apr", "04").when(col("month") == "May", "05").when(col("month") == "Jun", "06").when(col("month") == "Jul", "07").when(col("month") == "Aug", "08").when(col("month") == "Sep", "09").when(col("month") == "Oct", "10").when(col("month") == "Nov", "10").when(col("month") == "Dec", "12").otherwise("Unknown"))
df1=df1.withColumn("Date",F.concat_ws("-",df1.year,df1.Month,df1.Day))
df1=df1.withColumn('Log_category',F.regexp_replace(F.regexp_extract(df1.value,'(cat=[a-zA-Z]+)',1),'cat=',""))
df1=df1.withColumn("Attack_type",F.regexp_replace(F.regexp_extract(df1.value,'(#011name=[a-zA-Z:\s]+)',1),'#011name=',""))
df1=df1.withColumn("Description",F.regexp_replace(F.regexp_extract(df1.value,'(011desc=[a-zA-Z\s\/.]+)',1),'011desc=',""))
df1=df1.withColumn("Seviority",F.regexp_replace(F.regexp_extract(df1.value,'(011sev=[\d]+)',1),'011sev=',""))
df1=df1.withColumn("Source_ip",F.regexp_replace(F.regexp_extract(df1.value,'(#011src=[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1),'#011src=',""))
df1=df1.withColumn("User_name",F.regexp_replace(F.regexp_extract(df1.value,'(011usrName=[a-zA-Z \s]+)',1),'011usrName=',""))
df1=df1.withColumn("Destination_ip",F.regexp_extract(F.regexp_extract(df1.value,'(#011target=[0-9 . ( ) a-zA-Z ]+)',1),'([0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1))
df1=df1.withColumn("Desitination_host",F.regexp_replace(F.regexp_replace(F.regexp_extract(df1.value,'(#011target=[0-9 . ( ) a-zA-Z ]+)',1),'([0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',""),'#011target=' ,""))
df1=df1.withColumn("Rule_message",F.regexp_replace(F.regexp_extract(df1.value,'(#011msg=[0-9 . \/ a-zA-Z ]+)',1),'#011msg=',""))
df1=df1.withColumn("Data_source_ip",F.regexp_replace(F.regexp_extract(df1.value,'(#011TrendMicroDsScannerIp=[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1),'#011TrendMicroDsScannerIp=',""))
df1=df1.withColumn("Destination_port",F.regexp_replace(F.regexp_extract(df1.value,'(TrendMicroDsTargetPortList=[\d*;]*\d*)',1),'TrendMicroDsTargetPortList=',""))
df1=df1.withColumn("DataSource_IP",F.regexp_replace(F.regexp_extract(df1.value,'(DataSource_IP:[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1),'DataSource_IP:',""))

df2=df1.drop('value','_c6','year','Day','Month')
df2.show()

+--------+----------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+--------------+-----------------+--------------------+--------------+--------------------+-------------+
|    Time|      Date|Data_source_host_name|Log_category|         Attack_type|         Description|Seviority|     Source_ip|User_name|Destination_ip|Desitination_host|        Rule_message|Data_source_ip|    Destination_port|DataSource_IP|
+--------+----------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+--------------+-----------------+--------------------+--------------+--------------------+-------------+
|07:33:21| 2022-06-1|     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|192.168.83.163|   System|  10.129.76.76|  (sidcpaymeng08)|The Agent/Applian...|  10.129.76.75|7001;17000;17001;...|10.204.52.158|
|07:39:01| 2022-06-2|     TM-JIO-Console-1|     

In [42]:
df2=df1.drop('value','_c6','year','Day','Month')
df2.show()

+--------+----------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+--------------+-----------------+--------------------+--------------+--------------------+-------------+
|    Time|      Date|Data_source_host_name|Log_category|         Attack_type|         Description|Seviority|     Source_ip|User_name|Destination_ip|Desitination_host|        Rule_message|Data_source_ip|    Destination_port|DataSource_IP|
+--------+----------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+--------------+-----------------+--------------------+--------------+--------------------+-------------+
|07:33:21| 2022-06-1|     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|192.168.83.163|   System|  10.129.76.76|  (sidcpaymeng08)|The Agent/Applian...|  10.129.76.75|7001;17000;17001;...|10.204.52.158|
|07:39:01| 2022-06-2|     TM-JIO-Console-1|     